## Base Imports

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [5]:
BATCH_SIZE = 32
RES_ESPACIAL = 64
CHANNELS = 3

label_list = []
for class_name in os.listdir("galaxies_INF395/train/"):
  label_list.append(class_name)
label_list.sort()

print(label_list)

['Disk_Edge-on_Boxy_Bulge', 'Disk_Edge-on_No_Bulge', 'Disk_Edge-on_Rounded_Bulge', 'Disk_Face-on_Loose_Spiral', 'Disk_Face-on_Medium_Spiral', 'Disk_Face-on_No_Spiral', 'Disk_Face-on_Tight_Spiral', 'Smooth_Cigar_shaped', 'Smooth_Completely_round', 'Smooth_in-between_round']


In [10]:
idx2classname = {}
for idx, class_name in enumerate(label_list):
    idx2classname[idx] = class_name

print(idx2classname)

{0: 'Disk_Edge-on_Boxy_Bulge', 1: 'Disk_Edge-on_No_Bulge', 2: 'Disk_Edge-on_Rounded_Bulge', 3: 'Disk_Face-on_Loose_Spiral', 4: 'Disk_Face-on_Medium_Spiral', 5: 'Disk_Face-on_No_Spiral', 6: 'Disk_Face-on_Tight_Spiral', 7: 'Smooth_Cigar_shaped', 8: 'Smooth_Completely_round', 9: 'Smooth_in-between_round'}


Cargar datos de entrenamiento, validacion y pruebas.

In [20]:
def load_data(path, batch_size, res):
    data = tf.keras.preprocessing.image_dataset_from_directory(
        path,
        labels="inferred",
        label_mode="categorical",
        color_mode="rgb",
        batch_size=batch_size,
        image_size=(res, res),
        shuffle=True,
        seed=123
    )
    return data

train_data = load_data("galaxies_INF395/train/", BATCH_SIZE, RES_ESPACIAL)
val_data = load_data("galaxies_INF395/val/", BATCH_SIZE, RES_ESPACIAL)
test_data = load_data("galaxies_INF395/test/", BATCH_SIZE, RES_ESPACIAL)

Found 10636 files belonging to 10 classes.
Found 3542 files belonging to 10 classes.
Found 3558 files belonging to 10 classes.


In [21]:
def normalize(images, label):
    images = tf.cast(images, tf.float32) / 255.0
    return images, label

train_data = train_data.map(normalize)
val_data = val_data.map(normalize)
test_data = test_data.map(normalize)

train_dataset =  train_data.cache()
val_dataset =  val_data.cache()
test_dataset  =  test_data.cache()

In [22]:
for x,y in train_data:
    print(x.shape)
    print(y.shape)
    break

(32, 64, 64, 3)
(32, 10)


In [30]:
num_neurons = 256
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(num_neurons, activation=tf.nn.relu),
    tf.keras.layers.Dense(num_neurons, activation=tf.nn.relu),
    tf.keras.layers.Dense(num_neurons, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

#model.summary()

In [34]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

with tf.device('/GPU:0'):
    model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5
333/333 [==============================] - 7s 20ms/step - loss: 1.3062 - accuracy: 0.5330 - val_loss: 1.4091 - val_accuracy: 0.4932
Epoch 2/5
333/333 [==============================] - 7s 21ms/step - loss: 1.2431 - accuracy: 0.5533 - val_loss: 1.3507 - val_accuracy: 0.5130
Epoch 3/5
333/333 [==============================] - 7s 20ms/step - loss: 1.2221 - accuracy: 0.5549 - val_loss: 1.4237 - val_accuracy: 0.4912
Epoch 4/5
333/333 [==============================] - 7s 19ms/step - loss: 1.1594 - accuracy: 0.5794 - val_loss: 1.3842 - val_accuracy: 0.5071
Epoch 5/5
333/333 [==============================] - 7s 21ms/step - loss: 1.1039 - accuracy: 0.6014 - val_loss: 1.3580 - val_accuracy: 0.5161


In [35]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
